In [31]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from pandas import ExcelWriter

In [2]:
df = pd.read_csv("ChartbustersParticipantsData\Data_Train.csv",infer_datetime_format=True,encoding='utf-8')
df_test = pd.read_csv("ChartbustersParticipantsData\Data_Test.csv",infer_datetime_format=True,encoding='utf-8')
df_test['Views']=None
df.Name = df.Name.str.extract('([0-9a-zA-Z ]*)')
df_test.Name = df_test.Name.str.extract('([0-9a-zA-Z ]*)')
data = pd.concat([df,df_test],axis=0,sort=False,ignore_index=True)
data.index = data['Unique_ID']

In [3]:
data.drop(columns=['Unique_ID','Country','Song_Name'],inplace=True)

In [4]:
data.Views = pd.to_numeric(data.Views)
for cols in ['Likes','Popularity']:
    data[cols] = data[cols].apply(lambda x: x.replace(",",''))

In [5]:
data[data['Likes'].str.isnumeric()==False].Likes.str[-1].unique()

array(['K', 'M'], dtype=object)

In [6]:
data[data['Popularity'].str.isnumeric()==False].Likes.str[-1].unique()

array(['K', 'M'], dtype=object)

In [7]:
data['Likes']=np.where(data['Likes'].str[-1]=='K',pd.to_numeric(data['Likes'].str[0:-1])*1000,data['Likes'])
data['Likes']=np.where(data['Likes'].str[-1]=='M',pd.to_numeric(data['Likes'].str[0:-1])*1000000,data['Likes'])
data['Popularity']=np.where(data['Popularity'].str[-1]=='M',pd.to_numeric(data['Popularity'].str[0:-1])*1000000,data['Popularity'])
data['Popularity']=np.where(data['Popularity'].str[-1]=='K',pd.to_numeric(data['Popularity'].str[0:-1])*1000,data['Popularity'])

In [8]:
data['Timestamp']= pd.to_datetime(data['Timestamp'])
latest = data['Timestamp'].max()
data['Timestamp'] = (latest - data['Timestamp'] ).astype('timedelta64[D]')

In [9]:
data.Timestamp = data.Timestamp.astype(int)
data.Likes = data.Likes.astype(int)
data.Popularity = data.Popularity.astype(int)

In [10]:
cat_cols = pd.get_dummies(data.select_dtypes(exclude=np.number))

In [11]:
data['Followers_Per_Likes'] = np.where(data.Likes>0,data.Followers/data.Likes,0)
data['Followers_Per_Comments'] = np.where(data.Comments>0,data.Followers/data.Comments,0)
data['Likes_Per_Comments'] = np.where(data.Comments>0,data.Likes/data.Comments,0)
data['Likes_Per_Timestamp'] = np.where(data.Timestamp>0,data.Likes/data.Timestamp,0)
data['Likes_Per_Popularity'] =  np.where(data.Popularity>0,data.Likes/data.Popularity,0)

In [12]:
data['Songs_Cnt'] = data["Name"].map(data["Name"].value_counts())
data['Mean_Popularity'] = data['Name'].map(data.groupby('Name').Popularity.mean())
data['Avg_Likes'] = data['Name'].map(data.groupby('Name').Likes.mean())

In [13]:
data.head()

,Name,Genre,Timestamp,Views,Comments,Likes,Popularity,Followers,Followers_Per_Likes,Followers_Per_Comments,Likes_Per_Comments,Likes_Per_Timestamp,Likes_Per_Popularity,Songs_Cnt,Mean_Popularity,Avg_Likes
Unique_ID,,,,,,,,,,,,,,,,
413890,Hardstyle,danceedm,331,14017.0,4,499,97,119563,239.605210,29890.750000,124.750000,1.507553,5.144330,1422,134.361463,768.758790
249453,Dj Aladdin,danceedm,980,1918.0,17,49,17,2141,43.693878,125.941176,2.882353,0.050000,2.882353,30,59.433333,486.866667
681116,Maxximize On Air,danceedm,1388,9668.0,11,312,91,22248,71.307692,2022.545455,28.363636,0.224784,3.428571,193,129.015544,445.461140
387253,GR6 EXPLODE,rbsoul,626,113036.0,2,2400,76,393655,164.022917,196827.500000,1200.000000,3.833866,31.578947,1747,96.614768,4090.314825
1428029,Tritonal,danceedm,890,110024.0,81,3031,699,201030,66.324645,2481.851852,37.419753,3.405618,4.336195,342,565.909357,2314.546784


In [14]:
from sklearn.preprocessing import LabelEncoder
lbl = LabelEncoder()
data['Genre'] = lbl.fit_transform(data['Genre'])
data['Name'] = lbl.fit_transform(data['Name'])

In [15]:
data['Zero_Likes'] = np.where(data.Likes==0,1,0)
data['Zero_Comments'] = np.where(data.Comments==0,1,0)
data['Zero_Popularity'] = np.where(data.Popularity==0,1,0)

In [16]:
data['LikesnPopularity'] = data['Likes'] * data['Popularity']
data['LikesnFollowers'] = data['Likes'] * data['Followers']

In [ ]:
import pandas_profiling
pandas_profiling.ProfileReport(data)

In [17]:
drop_cols = ["Name_"+ name for name in df.Name.unique().tolist() if name not in df_test.Name.unique().tolist()]
cat_cols.drop(columns = drop_cols,inplace=True)

In [18]:
X = pd.concat([cat_cols,data.select_dtypes(include=np.number)],axis=1)

In [19]:
testX = X[X['Views'].isna()]
trainX = X[X['Views'].notna()]
trainY = trainX.pop('Views').astype(int)
testX.pop('Views')

Unique_ID
562546    NaN
907584    NaN
213013    NaN
340312    NaN
41854     NaN
           ..
183403    NaN
1272578   NaN
539454    NaN
1543013   NaN
415046    NaN
Name: Views, Length: 19615, dtype: float64

In [20]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso,LassoCV, Ridge, RidgeCV
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor

In [21]:
train_x, test_x, train_y,test_y = train_test_split(trainX,trainY,test_size=0.2, random_state=5)

In [22]:
print(train_x.shape,train_y.shape,test_x.shape,test_y.shape)

(62766, 1118) (62766,) (15692, 1118) (15692,)


In [193]:
lin = LassoCV(n_alphas=['0.1','1','10','0.01'])

In [168]:
lin = Lasso(alpha=0.1)
lin.fit(train_x,train_y)

C:\Users\hp\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1.1054460600635872e+16, tolerance: 90868133607135.53
  positive)


Lasso(alpha=0.1, copy_X=True, fit_intercept=True, max_iter=1000,
      normalize=False, positive=False, precompute=False, random_state=None,
      selection='cyclic', tol=0.0001, warm_start=False)

In [170]:
print("Train RMSE:",np.sqrt(mean_squared_error(train_y,lin.predict(train_x))))

Train RMSE: 593446.4156664688


In [169]:
print("Test RMSE:",np.sqrt(mean_squared_error(test_y,lin.predict(test_x))))

Test RMSE: 776949.3015830024


In [320]:
train_x.Timestamp.astype(int)

Unique_ID
212750     1299
574389      501
662009      999
962549        1
530696      489
           ... 
220974      676
1321506     216
448409     1877
212546      717
972679       19
Name: Timestamp, Length: 62766, dtype: int32

In [27]:
def model_fit(model,trainx,trainy,testx,testy):
    model.fit(trainx,trainy)
    trainypred = model.predict(trainx)
    print("Train RMSE:",np.sqrt(mean_squared_error(trainy,trainypred)))
    testypred = model.predict(testx)
    print("Test RMSE:",np.sqrt(mean_squared_error(testy,testypred)))
    #return testypred
    

In [28]:
def model_predict(model,trainx,trainy,testx):
    model.fit(trainx,trainy)
    trainypred = model.predict(trainx)
    print("Train RMSE:",np.sqrt(mean_squared_error(trainy,trainypred)))
    testypred = model.predict(testx)
    return testypred
    

In [29]:
def write_results(ytest,file):
    result_df= pd.DataFrame([ytest]).T
    result_df.columns=['Views']
    result_df.index = testX.index
    writer = ExcelWriter(file+'.xlsx')
    result_df.to_excel(writer,index=True)
    writer.save()

In [59]:
rfr = RandomForestRegressor(random_state=5,n_estimators=5,max_features=0.8)

In [60]:
model_fit(rfr,train_x,train_y,test_x,test_y)

Train RMSE: 352498.2981552757
Test RMSE: 742810.6180487007


In [270]:
write_results(ytest,"RandomForest")

In [58]:
ytest = model_predict(rfr,trainX,trainY,testX)
write_results(ytest,"RandomForest10")

Train RMSE: 339342.3482312637


In [203]:
for i in range(10,100,10):
    rfr = RandomForestRegressor(random_state=5,n_estimators=i)
    print("n_estimators ={} :".format(i))
    model_fit(rfr,train_x,train_y,test_x,test_y)
    

n_estimators =10 :
Train RMSE: 338793.64673769096
Test RMSE: 747242.148907676
n_estimators =20 :
Train RMSE: 313754.0732218443
Test RMSE: 723959.5011887817
n_estimators =30 :


KeyboardInterrupt: 

In [61]:
feat_imp = rfr.feature_importances_
feat_df = pd.concat([pd.DataFrame(testX.columns),pd.DataFrame(feat_imp)],axis=1)
feat_df.columns=['Column','Importance']
imp_cols=feat_df.sort_values(by=['Importance'],ascending=False)['Column'].head(14)

In [157]:
for i in range(10,20,2):
    rfr = RandomForestRegressor(random_state=5,n_estimators=10,max_features=0.8)
    imp_cols=feat_df.sort_values(by=['Importance'],ascending=False)['Column'].head(i)
    rfr.fit(train_x[imp_cols],train_y)
    testypred=rfr.predict(test_x[imp_cols])
    print("i={}:,Test RMSE:{}".format(i,np.sqrt(mean_squared_error(test_y,testypred))))
    

i=10:,Test RMSE:792109.1068301828
i=12:,Test RMSE:778333.89230019
i=14:,Test RMSE:718245.7001436497
i=16:,Test RMSE:726401.3098877755
i=18:,Test RMSE:788936.0394901263


In [63]:
imp_cols=feat_df.sort_values(by=['Importance'],ascending=False)['Column'].head(14)
rfr = RandomForestRegressor(random_state=5,n_estimators=100,max_features=0.8)
ytest = model_predict(rfr,trainX[imp_cols],trainY,testX[imp_cols])
write_results(ytest,"RandomForest_feat14")

Train RMSE: 256934.25376932765


In [187]:
x_scaled = MinMaxScaler().fit_transform(X[imp_cols])

In [188]:
train_x,test_x,train_y,test_y = train_test_split(x_scaled[:trainX.shape[0],:],trainY,test_size=0.2,random_state=5)

In [189]:
train_x.shape

(62766, 20)

In [190]:
train_y.shape

(62766,)

In [335]:
feat_df.sort_values(by=['Importance'],ascending=False)

,Column,Importance
1099,Comments,0.227380
1101,Popularity,0.192404
1100,Likes,0.149666
1110,Likes_Per_Timestamp,0.086726
1111,LikesnPopularity,0.065085
...,...,...
556,Name_Marvel Boy,0.000000
633,Name_Nu Elect Remix,0.000000
689,Name_Prince Winston Thee Third,0.000000
882,Name_Tuan Lux,0.000000


In [61]:
knn = KNeighborsRegressor(n_neighbors=5)

In [192]:
for i in range(100,120,10):
    knn = KNeighborsRegressor(n_neighbors=i)
    #imp_cols=feat_df.sort_values(by=['Importance'],ascending=False)['Column'].head(60)
    knn.fit(train_x[imp_cols],train_y)
    testypred=knn.predict(test_x[imp_cols])
    print("i={}:,Test RMSE:{}".format(i,np.sqrt(mean_squared_error(test_y,testypred))))

i=250:,Test RMSE:3268262.0628176234


KeyboardInterrupt: 

In [70]:
from xgboost import XGBRegressor

In [71]:
xgb = XGBRegressor()

In [76]:
trainX['Name_Major Lazer'] = trainX['Name_Major Lazer [OFFICIAL]'

C:\Users\hp\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [340]:

xgb = XGBRegressor(random_state=5,n_estimators=200)
ytest = model_predict(xgb,trainX[imp_cols],trainY,testX[imp_cols])
write_results(ytest,"XGBoost3")

C:\Users\hp\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\hp\Anaconda3\lib\site-packages\xgboost\core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


[04:06:39] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Train RMSE: 328611.71695205546


In [179]:
for i in range(4,7):
    xgb = XGBRegressor(random_state=5,n_estimators=500,max_depth=i)
    xgb.fit(train_x[imp_cols],train_y)
    testypred=xgb.predict(test_x[imp_cols])
    print("i={}:,Test RMSE:{}".format(i,np.sqrt(mean_squared_error(test_y,testypred))))
    
    

C:\Users\hp\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[01:28:05] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
i=4:,Test RMSE:681832.7582993128


C:\Users\hp\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[01:29:38] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
i=5:,Test RMSE:648465.7880878866


C:\Users\hp\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[01:31:26] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
i=6:,Test RMSE:640780.9182854426


In [167]:
feat_df[feat_df.Importance>0].sort_values(by=['Importance'],ascending=False).head(60)

,Column,Importance
1102,Likes,0.805450
1103,Popularity,0.138274
1101,Comments,0.019616
1109,Likes_Per_Popularity,0.005830
1082,Genre_danceedm,0.003450
1108,Likes_Per_Timestamp,0.003131
74,Name_BTS,0.003109
1106,Followers_Per_Comments,0.002999
1117,LikesnFollowers,0.002741
1116,LikesnPopularity,0.002095


In [182]:
np.sqrt(62766)

250.53143515335555

In [23]:
from sklearn.tree import DecisionTreeRegressor

In [50]:
dtr = DecisionTreeRegressor(random_state=5,max_features=0.6,max_depth=25, min_samples_split=8, min_samples_leaf=1)

In [51]:
ytest = model_predict(dtree,trainX,trainY,testX)
write_results(ytest,"DecisionTree")

Train RMSE: 380.687606461636


In [166]:
for i in range(1,2):
    dtr = DecisionTreeRegressor(random_state=5,max_depth=50,min_samples_split=5,min_samples_leaf=1)
    imp_cols=feat_df.sort_values(by=['Importance'],ascending=False)['Column'].head(7)
    dtr.fit(train_x,train_y)
    testypred=dtr.predict(test_x)
    print("i={}:,Test RMSE:{}".format(i,np.sqrt(mean_squared_error(test_y,testypred))))

i=1:,Test RMSE:1021613.2274996121


In [158]:
dtr = DecisionTreeRegressor(random_state=5,max_depth=5,min_samples_split=9,min_samples_leaf=1)
imp_cols=feat_df.sort_values(by=['Importance'],ascending=False)['Column'].head(7)
ytest = model_predict(dtr,trainX[imp_cols],trainY,testX[imp_cols])
write_results(ytest,"DecisionTree7")

Train RMSE: 627042.4391973766


In [208]:
from sklearn.ensemble import VotingRegressor

In [211]:
dtree = DecisionTreeRegressor(random_state=5,max_features=20)
rfr = RandomForestRegressor(random_state=5,n_estimators=40,max_features=60)
voting = VotingRegressor(estimators=[
('DTree',dtree),('RanFor',rfr)
])

In [212]:
ytest = model_predict(voting,trainX[imp_cols],trainY,testX[imp_cols])
write_results(ytest,"Voting")

Train RMSE: 153981.5900797446


In [172]:
from sklearn.ensemble import AdaBoostRegressor, GradientBoostingRegressor

In [173]:
ada = AdaBoostRegressor(random_state=5)

In [175]:
imp_cols=feat_df.sort_values(by=['Importance'],ascending=False)['Column'].head(20)
ytest = model_predict(ada,trainX[imp_cols],trainY,testX[imp_cols])
write_results(ytest,"AdaBoost")

Train RMSE: 1329661.1885196608


In [185]:
imp_cols=feat_df.sort_values(by=['Importance'],ascending=False)['Column'].head(200)
gbr = GradientBoostingRegressor(random_state=5,learning_rate=0.1,n_estimators=90)

In [186]:
ytest = model_predict(gbr,trainX[imp_cols],trainY,testX[imp_cols])
write_results(ytest,"GradientBoost")

Train RMSE: 371090.3164493857


In [187]:
from sklearn.model_selection import StratifiedKFold,cross_val_score

In [188]:
skf = StratifiedKFold(n_splits=10)

In [ ]:
cross_val_score(gbr,trainX[imp_cols],trainY, cv=skf)

C:\Users\hp\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:657: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)


In [184]:
for i in range(50,100,10):
    dtr = GradientBoostingRegressor(random_state=5,learning_rate=0.1,n_estimators=i)
    imp_cols=feat_df.sort_values(by=['Importance'],ascending=False)['Column'].head(200)
    dtr.fit(train_x[imp_cols],train_y)
    testypred=dtr.predict(test_x[imp_cols])
    print("i={}:,Test RMSE:{}".format(i,np.sqrt(mean_squared_error(test_y,testypred))))

i=50:,Test RMSE:709550.7984036658
i=60:,Test RMSE:700352.4043052795
i=70:,Test RMSE:685292.8319867053
i=80:,Test RMSE:676308.8682145509
i=90:,Test RMSE:669363.6439652127


In [224]:
from sklearn.linear_model

982              Popularity
981                   Likes
993    Likes_Per_Popularity
991     Likes_Per_Timestamp
986      Likes_Per_Comments
               ...         
224               Name_Eric
804            Name_Virtual
317            Name_Hussein
618              Name_ROJAS
42                Name_Amin
Name: Column, Length: 200, dtype: object

In [228]:
from sklearn.preprocessing import PolynomialFeatures

In [229]:
poly = PolynomialFeatures(degree=2)

In [246]:
imp_cols=feat_df.sort_values(by=['Importance'],ascending=False)['Column'].head(20)
trainXpoly = poly.fit_transform(trainX[imp_cols])

In [247]:
testXpoly = poly.transform(testX[imp_cols])

In [234]:
lin = LinearRegression()

In [238]:
ytest = model_predict(rfr,trainXpoly,trainY,testXpoly)
write_results(ytest,"Polynomial_RFR")

Train RMSE: 289047.50823923585


In [241]:
imp_cols=feat_df.sort_values(by=['Importance'],ascending=False)['Column'].head(20)
DT = DecisionTreeRegressor(random_state=5, max_features=20)
DT.fit(trainX[imp_cols],trainY)


DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=20,
                      max_leaf_nodes=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      presort=False, random_state=5, splitter='best')

In [242]:
testy_DT = DT.predict(testX[imp_cols])

In [243]:
rfr = RandomForestRegressor(random_state=5,n_estimators=40)

In [244]:
imp_cols=feat_df.sort_values(by=['Importance'],ascending=False)['Column'].head(60)
rfr.fit(trainX[imp_cols],trainY)
testy_RF = rfr.predict(testX[imp_cols])

In [248]:
gbr= GradientBoostingRegressor(random_state=5)
gbr.fit(trainXpoly,trainY)
testy_GBR = rfr.predict(testXpoly)

ValueError: Number of features of the model must match the input. Model n_features is 60 and input n_features is 231 

In [250]:
ensemble = np.mean([testy_DT
                              ,testy_RF
                             ], axis = 0)

In [251]:
ensemble

array([214042.9875,  30803.2875,  15332.225 , ...,   4142.    ,
       101363.35  ,  21217.4875])

In [253]:
result_df = pd.DataFrame(ensemble)

In [254]:
result_df.columns=['Views']
result_df.index = testX.index

In [257]:
writer = ExcelWriter('MeanTree.xlsx')
result_df.to_excel(writer,index=True)

In [258]:
result_df

,Views
Unique_ID,
562546,214042.9875
907584,30803.2875
213013,15332.2250
340312,32708.9875
41854,8542.1875
...,...
183403,65378.4500
1272578,99054.7500
539454,4142.0000


In [259]:
rfr = RandomForestRegressor(random_state=5,n_estimators=1000,max_features=0.1)
ytest = model_predict(rfr,trainX,trainY,testX)
write_results(ytest,"RFR")

KeyboardInterrupt: 

In [356]:
xgb1 = XGBRegressor()
parameters = {'nthread':[2], #when use hyperthread, xgboost may become slower
              'objective':['reg:linear'],
              'learning_rate': [.03, 0.05, .07], #so called `eta` value
              'max_depth': [5, 6, 7],
              'min_child_weight': [4],
              'silent': [1],
              'subsample': [0.7],
              'colsample_bytree': [0.7],
              'n_estimators': [100,200,500]}

xgb_grid = GridSearchCV(xgb1,
                        parameters,
                        cv = 5,
                        n_jobs = 2,
                        verbose=True)

xgb_grid.fit(trainX[imp_cols],
         trainY)

print(xgb_grid.best_score_)
print(xgb_grid.best_params_)

Fitting 5 folds for each of 27 candidates, totalling 135 fits


[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed: 16.4min
[Parallel(n_jobs=2)]: Done 135 out of 135 | elapsed: 46.3min finished
C:\Users\hp\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\hp\Anaconda3\lib\site-packages\xgboost\core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


0.9692657134175081
{'colsample_bytree': 0.7, 'learning_rate': 0.07, 'max_depth': 5, 'min_child_weight': 4, 'n_estimators': 500, 'nthread': 2, 'objective': 'reg:linear', 'silent': 1, 'subsample': 0.7}


In [354]:
from sklearn.model_selection import GridSearchCV
from xgboost import XGBRegressor
xgbr = XGBRegressor(colsample_bytree= 0.7, learning_rate= 0.07, max_depth=5, min_child_weight= 4, n_estimators= 500, nthread= 2, objective= 'reg:linear', silent= 1, subsample= 0.7)

In [133]:
from xgboost import XGBRegressor
xgbr = XGBRegressor(random_state=5,colsample_bytree= 0.7, max_depth=8, min_child_weight= 4, n_estimators= 500, nthread= 2, objective= 'reg:linear', silent= 1, subsample= 0.7)

In [134]:
xgbr.fit(train_x,train_y,eval_set=[(test_x,test_y)],early_stopping_rounds=5, eval_metric="rmse",verbose=10)

C:\Users\hp\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[0]	validation_0-rmse:3.83727e+06
Will train until validation_0-rmse hasn't improved in 5 rounds.
[10]	validation_0-rmse:1.61809e+06
[20]	validation_0-rmse:953975
[30]	validation_0-rmse:781510
[40]	validation_0-rmse:724023
[50]	validation_0-rmse:701519
[60]	validation_0-rmse:694806
Stopping. Best iteration:
[59]	validation_0-rmse:694737



XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.7, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=8, min_child_weight=4, missing=None, n_estimators=500,
             n_jobs=1, nthread=2, objective='reg:linear', random_state=5,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None, silent=1,
             subsample=0.7, verbosity=1)

In [135]:
xgbr = XGBRegressor(n_estimators=59, random_state=5, max_depth=8, min_child_weight= 4, nthread= 2, objective= 'reg:linear', silent= 1)
testy = model_predict(xgbr,trainX,trainY,testX)
# make predictions for test data

C:\Users\hp\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\hp\Anaconda3\lib\site-packages\xgboost\core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


Train RMSE: 177359.4054652234


In [138]:
predictions = [round(value) if value>0 else 0 for value in testy ]
write_results(predictions,"XGBRGridCV2.xlsx")

In [137]:
n= [value for value in testy if value<0 ]
n

[-4388.994]

In [115]:
import lightgbm as lgb
lgb_fit_params={"early_stopping_rounds":100, 
            "eval_metric" : 'rmse', 
            "eval_set" : [(test_x,test_y)],
            'eval_names': ['valid'],
            'verbose':100
           }

lgb_params = {'boosting_type': 'gbdt',
 'metric': 'rmse',
 'verbose': 0,
 'lambda_l1': 0.01,
 'lambda_l2': 0.01,
 'bagging_freq':1,
 'bagging_fraction':0.8,
 'learning_rate': 0.1,
 'max_depth': -1,
 'feature_fraction':0.6,
 'min_data_in_bin': 1,
 'min_data_in_leaf': 1,
 'num_leaves': 31}
lgb_params

clf_lgb = lgb.LGBMRegressor(n_estimators=1000, **lgb_params, random_state=5, n_jobs=2)
clf_lgb.fit(train_x, train_y, **lgb_fit_params)
clf_lgb.best_iteration_
clf_lgb.best_score_

Training until validation scores don't improve for 100 rounds
[100]	valid's rmse: 1.03788e+06
[200]	valid's rmse: 1.02805e+06
[300]	valid's rmse: 1.02338e+06
[400]	valid's rmse: 1.02198e+06
[500]	valid's rmse: 1.02099e+06
[600]	valid's rmse: 1.02012e+06
[700]	valid's rmse: 1.02002e+06
[800]	valid's rmse: 1.01993e+06
[900]	valid's rmse: 1.01989e+06
Early stopping, best iteration is:
[851]	valid's rmse: 1.01977e+06


defaultdict(collections.OrderedDict,
            {'valid': OrderedDict([('rmse', 1019774.4363405305)])})

In [393]:
clf_lgb_full = lgb.LGBMRegressor(n_estimators=int(clf_lgb.best_iteration_*1.2), **lgb_params, random_state=5, n_jobs=2)
clf_lgb_full.fit(trainX, trainY)

LGBMRegressor(bagging_fraction=0.8, bagging_freq=1, boosting_type='gbdt',
              class_weight=None, colsample_bytree=1.0, feature_fraction=0.7,
              importance_type='split', lambda_l1=0.01, lambda_l2=0.01,
              learning_rate=0.1, max_bin=255, max_depth=-1, metric='rmse',
              min_child_samples=20, min_child_weight=0.001, min_data_in_bin=1,
              min_data_in_leaf=1, min_split_gain=0.0, n_estimators=508,
              n_jobs=2, num_leaves=31, objective=None, random_state=5,
              reg_alpha=0.0, reg_lambda=0.0, silent=True, subsample=1.0,
              subsample_for_bin=200000, subsample_freq=0, verbose=0)

In [395]:
ytest = model_predict(clf_lgb_full,trainX[imp_cols],trainY,testX[imp_cols])
write_results(ytest,"LightGBMR")

Train RMSE: 72964.70008252536


In [143]:
feat_df.sort_values(by=['Importance'],ascending=False)

,Column,Importance
1102,Likes,0.805450
1103,Popularity,0.138274
1101,Comments,0.019616
1109,Likes_Per_Popularity,0.005830
1082,Genre_danceedm,0.003450
...,...,...
947,Name_YG Entertainment,0.000000
474,Name_Kyle Murdock 1,0.000000
468,Name_Kora,0.000000
816,Name_TCT Production,0.000000
